In [1]:
print ("Hola Mundo")

Hola Mundo


Hola Mundo

In [ ]:
df_titanic = pd.read_csv("./data/titanic_modified.csv")

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def get_features_cat_regression(dataframe, target_col, pvalue=0.05):
    """
    Identifica columnas categóricas (numéricas con baja cardinalidad o explícitamente categóricas)
    que tienen una relación significativa con una columna numérica objetivo usando la prueba de Chi-cuadrado.

    Parámetros:
    - dataframe: pd.DataFrame. El DataFrame de entrada.
    - target_col: str. Nombre de la columna objetivo, debe ser numérica.
    - pvalue: float. Nivel de significancia para la prueba Chi-cuadrado.

    Retorna:
    - Una lista de columnas categóricas significativas o None si no se encuentran.
    """
    # Dividir columnas explícitamente categóricas y numéricas
    explicit_categorical_cols = [col for col in dataframe.select_dtypes(include=['object', 'category']).columns if col != target_col]
    potential_categorical_cols = [col for col in dataframe.select_dtypes(include=[np.number]).columns
                                   if col != target_col and dataframe[col].nunique() <= 10]

    # Combinar ambas listas
    all_categorical_cols = explicit_categorical_cols + potential_categorical_cols

    if not all_categorical_cols:
        print("Error: No se encontraron columnas categóricas o numéricas con baja cardinalidad.")
        return None

    significant_features = []

    for col in all_categorical_cols:
        try:
            # Crear una tabla de contingencia
            contingency_table = pd.crosstab(dataframe[col], pd.cut(dataframe[target_col], bins=5))

            # Realizar la prueba de Chi-cuadrado
            chi2, p, dof, expected = chi2_contingency(contingency_table)

            # Comprobar si el p-valor es menor que el umbral dado
            if p < pvalue:
                significant_features.append(col)

        except Exception as e:
            print(f"Advertencia: Error al procesar la columna '{col}': {e}")

    if not significant_features:
        print("No se encontraron columnas categóricas con una relación significativa con el target.")
        return []

    return significant_features

In [ ]:
get_features_cat_regression(df_titanic, target_col="Survived")

In [ ]:
columns = ['Pclass', 'Sex', 'SibSp', 'Parch', 'MissingAge', 'Spouse']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_features_cat_regression(dataframe, target_col, categorical_columns, with_individual_plot=False):
    """
    Dibuja histogramas de la relación entre las columnas categóricas y la columna objetivo
    utilizando subplots para mostrar todas las gráficas juntas en una cuadrícula.

    Parámetros:
    - dataframe: pd.DataFrame. El DataFrame con los datos.
    - target_col: str. Nombre de la columna objetivo numérica.
    - categorical_columns: list. Lista de columnas categóricas significativas obtenidas.
    - with_individual_plot: bool. Si True, genera solo gráficos individuales para cada columna.

    Retorna:
    - None. Dibuja los histogramas en pantalla.
    """
    if not categorical_columns:
        print("No se proporcionaron columnas categóricas para graficar.")
        return

    # Si se solicita gráficos individuales, no crear subplots
    if with_individual_plot:
        for col in categorical_columns:
            try:
                plt.figure(figsize=(10, 6))
                sns.countplot(
                    data=dataframe,
                    x=target_col,
                    hue=col,
                    palette="tab10"
                )
                plt.title(f"Histograma individual de '{target_col}' agrupado por '{col}'", fontsize=14)
                plt.xlabel(target_col, fontsize=12)
                plt.ylabel("Frecuencia", fontsize=12)
                plt.xticks(ticks=[0, 1], labels=["0", "1"], fontsize=10)
                plt.grid(axis='y', linestyle='--', alpha=0.7)
                plt.legend(title=col, fontsize=10)
                plt.tight_layout()
                plt.show()
            except Exception as e:
                print(f"Error al graficar la columna '{col}': {e}")
        return  # Salir de la función tras generar gráficos individuales

    # Crear subplots si no se solicitan gráficos individuales
    num_cols = len(categorical_columns)
    cols_per_row = 2  # Número de columnas por fila
    rows = (num_cols + cols_per_row - 1) // cols_per_row  # Calcular el número de filas necesarias

    fig, axes = plt.subplots(nrows=rows, ncols=cols_per_row, figsize=(15, 6 * rows), constrained_layout=True)
    axes = axes.flatten()  # Aplanar la matriz de ejes para iterar fácilmente

    for i, col in enumerate(categorical_columns):
        try:
            sns.countplot(
                data=dataframe,
                x=target_col,
                hue=col,
                palette="tab10",
                ax=axes[i]
            )
            axes[i].set_title(f"Histograma de '{target_col}' agrupado por '{col}'", fontsize=14)
            axes[i].set_xlabel(target_col, fontsize=12)
            axes[i].set_ylabel("Frecuencia", fontsize=12)
            
            axes[i].grid(axis='y', linestyle='--', alpha=0.7)
            axes[i].legend(title=col, fontsize=10)

        except Exception as e:
            print(f"Error al graficar la columna '{col}': {e}")

    # Eliminar ejes vacíos si hay menos subplots que espacios disponibles
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.show()

In [ ]:
from toolbox_ML import plot_features_cat_regression 

In [ ]:
plot_features_cat_regression(df_titanic,target_col="Survived",categorical_columns= columns)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np

def filter_and_plot_features(dataframe, target_col="", columns=None, pvalue=0.05, with_individual_plot=False):
    """
    Filtra y grafica columnas categóricas que tienen una relación significativa con una columna objetivo.

    Parámetros:
    - dataframe: pd.DataFrame. El DataFrame de entrada.
    - target_col: str. Nombre de la columna objetivo.
    - columns: list. Lista de columnas categóricas a evaluar. Si está vacía, se seleccionan automáticamente.
    - pvalue: float. Nivel de significancia para la prueba Chi-cuadrado.
    - with_individual_plot: bool. Si True, genera gráficos individuales para cada columna significativa.

    Retorna:
    - Una lista de columnas que cumplen con las condiciones de significancia.
    """
    if not isinstance(dataframe, pd.DataFrame):
        print("Error: El argumento 'dataframe' debe ser un DataFrame de pandas.")
        return None

    if not target_col or target_col not in dataframe.columns:
        print("Error: La columna objetivo no existe o no se especificó correctamente.")
        return None

    if not np.issubdtype(dataframe[target_col].dtype, np.number):
        print("Error: La columna objetivo debe ser numérica.")
        return None

    if columns is None:
        columns = []

    if not columns:
        # Seleccionar columnas numéricas con baja cardinalidad
        columns = [col for col in dataframe.select_dtypes(include=[np.number]).columns 
                   if col != target_col and dataframe[col].nunique() <= 10]

    if not columns:
        print("No se encontraron columnas para evaluar.")
        return []

    significant_columns = []

    for col in columns:
        try:
            # Crear tabla de contingencia
            contingency_table = pd.crosstab(dataframe[col], pd.cut(dataframe[target_col], bins=5))

            # Realizar prueba de Chi-cuadrado
            chi2, p, dof, expected = chi2_contingency(contingency_table)

            # Evaluar significancia
            if p < pvalue:
                significant_columns.append(col)

                # Graficar si se solicita
                if with_individual_plot:
                    plt.figure(figsize=(10, 6))
                    sns.countplot(
                        data=dataframe,
                        x=target_col,
                        hue=col,
                        palette="tab10"
                    )
                    plt.title(f"Histograma de '{target_col}' agrupado por '{col}'", fontsize=14)
                    plt.xlabel(target_col, fontsize=12)
                    plt.ylabel("Frecuencia", fontsize=12)
                    plt.grid(axis='y', linestyle='--', alpha=0.7)
                    plt.legend(title=col, fontsize=10)
                    plt.tight_layout()
                    plt.show()

        except Exception as e:
            print(f"Error al procesar la columna '{col}': {e}")

    # Graficar todas las columnas significativas juntas
    if significant_columns:
        num_cols = len(significant_columns)
        cols_per_row = 2  # Número de columnas por fila
        rows = (num_cols + cols_per_row - 1) // cols_per_row

        fig, axes = plt.subplots(nrows=rows, ncols=cols_per_row, figsize=(15, 6 * rows), constrained_layout=True)
        axes = axes.flatten()

        for i, col in enumerate(significant_columns):
            sns.countplot(
                data=dataframe,
                x=target_col,
                hue=col,
                palette="tab10",
                ax=axes[i]
            )
            axes[i].set_title(f"Histograma de '{target_col}' agrupado por '{col}'", fontsize=14)
            axes[i].set_xlabel(target_col, fontsize=12)
            axes[i].set_ylabel("Frecuencia", fontsize=12)
            
            axes[i].grid(axis='y', linestyle='--', alpha=0.7)
            axes[i].legend(title=col, fontsize=10)

        # Eliminar subplots vacíos
        for j in range(len(significant_columns), len(axes)):
            fig.delaxes(axes[j])

        plt.show()

    return significant_columns


In [ ]:
filter_and_plot_features(df_titanic, target_col="Survived")